In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import xml.etree.ElementTree as ET
import json
import requests
import urllib
import urllib.request
import datetime
import folium
import warnings
import lightgbm
import xgboost
from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [3]:
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

a = ['토마토']
code_dict={
    '전라남도 해남군': '536824B002',
    '제주도 제주시': '063057B009',
    '전라남도 무안군': '534833E001',
    '전남 해남군': '536824B002',
    '경상남도 창녕군': '635821A001',
    '전라남도 진도군': '539823A001',
    '경기도 포천시': '487915A001',
    '경상남도 밀양시': '627911A001',
    '경기도 시흥시': '429843A001',
    '강원도 철원군': '269811A001',
    '충청남도 천안시': '330846A001',
    '경상북도 청도군': '714902A001',
    '부산 강서구': '618803A001',
    '전북 남원시': '590823A001',
    '경상남도 진주시': '660985B001',
    '경상북도 상주시': '742290A001',
    '경상북도 김천시': '037268B004'
}

joosan_dict={
    '배추': '전라남도 해남군',
    '무': '제주도 제주시',
    '양파': '전라남도 무안군',
    '건고추': '전남 해남군',
    '마늘': '경상남도 창녕군',
    '대파': '전라남도 진도군',
    '얼갈이배추': '경기도 포천시',
    '양배추': '제주도 제주시',
    '깻잎': '경상남도 밀양시',
    '시금치': '경기도 포천시',
    '미나리': '경기도 시흥시',
    '당근': '제주도 제주시',
    '파프리카': '강원도 철원군',
    '새송이': '충청남도 천안시',
    '팽이버섯': '경상북도 청도군',
    '토마토': '부산 강서구',
    '청상추': '전북 남원시',
    '백다다기': '충청남도 천안시',
    '애호박': '경상남도 진주시',
    '캠벨얼리': '경상북도 상주시',
    '샤인마스캇': '경상북도 김천시'
    }

In [4]:
weather=pd.read_csv('/content/drive/Shareddrives/마켓컬리_해커톤/data/weather.csv')
weather

,no,stn_Code,stn_Name,date,temp,max_Temp,min_Temp,hum,widdir,wind,rain,sun_Time,sun_Qy,condens_Time,gr_Temp,soil_Temp,soil_Wt
0,1,536824B002,해남군 옥천면,2015-01-01,-1.3,0.6,-2.9,80.0,295.2,2.3,0.8,NaN,7.8,NaN,NaN,3.36,25.9
1,1,330846A001,천안시 목천읍,2015-01-01,-6.2,-3.8,-8.3,NaN,NaN,0.0,0.0,NaN,NaN,1429.0,NaN,NaN,NaN
2,1,627911A001,밀양시 상남면,2015-01-01,-3.2,0.2,-7.2,40.1,282.7,2.9,0.0,516.0,11.0,0.0,NaN,2.20,28.5
3,1,539823A001,진도군 군내면,2015-01-01,-0.8,1.6,-2.8,79.2,257.0,3.5,1.5,217.0,8.2,652.0,NaN,5.02,30.6
4,1,590823A001,남원시 이백면,2015-01-01,-4.1,-1.3,-6.0,60.7,286.7,2.1,0.5,310.0,7.7,0.0,-4.3,2.16,20.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34258,31,590823A001,남원시 이백면,2020-12-31,-2.6,-0.3,-5.8,74.6,232.0,1.0,0.0,339.0,9.7,87.0,-2.6,7.23,22.8
34259,31,534833E001,무안군 청계면,2020-12-31,-3.7,0.9,-8.3,95.3,353.5,1.5,1.0,215.0,8.2,0.0,-1.2,2.50,18.6
34260,31,330846A001,천안시 목천읍,2020-12-31,-9.9,-4.2,-16.4,70.3,171.9,0.1,0.5,NaN,30.4,0.0,-10.4,1.03,13.3
34261,31,269811A001,철원군 동송읍,2020-12-31,-11.2,-5.5,-16.9,43.0,250.6,2.3,0.0,NaN,8.3,0.0,-5.7,-4.95,22.5


In [5]:
def preprocessing1(temp_df,train, pum, len_lag, weather, joosan_dict, code_dict) :
    weather_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    # 품종과 주산지 날씨 mapping
    
    joosanji = joosan_dict[pum]
    joosan_code = code_dict[joosanji]
    joosan_weather = weather[weather['stn_Code']==joosan_code].reset_index(drop=True)
    
    end_index = np.where(joosan_weather['date']=='2020-09-28')[0][0]
    joosan_weather = joosan_weather.iloc[:end_index+1] #2020-09-28 까지만 자르기
    weather_df = weather_df.merge(joosan_weather, on='date', how='left')
    
    # 해당하는 열의 모든 값이 null 값이기 때문에 이열 두개는 삭제 처리  
    weather_df.drop(['condens_Time','gr_Temp'],axis=1,inplace=True)
    # 결측치가 존재함 -> 날씨의 특성상 앞뒤 날씨의 영향을 받거나 혹은 비슷한 시기의 날씨의 특성과 비슷할 것으로 예상
    # 따라서 결측치를 앞날의 데이터를 가져오는 것으로 결정 
    weather_df['sun_Time']=weather_df['sun_Time'].fillna(method='ffill')
    weather_df['soil_Wt']=weather_df['soil_Wt'].fillna(method='ffill')
    wea_col_list=list(weather_df.columns)
    del wea_col_list[0:6]
    #print(weather_df)
    
    # p_lag, q_lag 추가, 날씨 열 추가
    for lag in range(1,len_lag+1) :
      temp_df[f'p_lag_{lag}'] = -1
      temp_df[f'q_lag_{lag}'] = -1
      for col in range(len(wea_col_list)):
        col=wea_col_list[col]
         #or col =="widdir" or col == "wind"
        if(col == "sun_Time" or col == "sun_Qy" or col =="soil_Temp" or col =="soil_Wt" or col == "wind" or col =="widdir"):
            continue
        if(pum == '건고추' or pum == '당근' or pum =='토마토'):
            continue
        temp_df[f'{col}_lag_{lag}'] = -1
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] #1일전, 2일전, ... 가격을 feature로 추가
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] #1일전, 2일전, ... 거래량을 feature로 추가
        if(pum == '건고추' or pum == '당근' or pum =='토마토'):
            continue
        temp_df.loc[index, f'temp_lag_{lag}'] = weather_df['temp'][index-lag]
        temp_df.loc[index, f'max_Temp_lag_{lag}'] = weather_df['max_Temp'][index-lag]
        temp_df.loc[index, f'min_Temp_lag_{lag}'] = weather_df['min_Temp'][index-lag]
        temp_df.loc[index, f'hum_lag_{lag}'] = weather_df['hum'][index-lag]
        temp_df.loc[index, f'rain_lag_{lag}'] = weather_df['rain'][index-lag]
   
    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['week']=temp_df['date'].dt.week
    temp_df['day'] = temp_df['date'].dt.weekday
 
    

    # 예측 대상(1w,2w,4w) 추가
    for week in ['1_week','2_week','4_week'] :
      temp_df[week] = 0
      n_week = int(week[0])
      for index in range(len(temp_df)) :
        try : temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
        except : continue

    # 불필요한 column 제거        
    temp_df = temp_df.drop(['date',f'{pum}_거래량(kg)',f'{pum}_가격(원/kg)'], axis=1)
    
    return temp_df

In [6]:
train = pd.read_csv('/content/drive/Shareddrives/마켓컬리_해커톤/data/train.csv')
#train = train.drop('Unnamed: 0', axis = 1)
train #2015.12 ~ 2020.09

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,2020-09-24,목요일,1856965.0,1839.0,2055640.0,990.0,2281429.2,990.0,2818.4,19101.0,...,50730.0,4509.0,282212.3,3001.0,313139.7,3426.0,504242.6,3620.0,283196.9,10940.0
1729,2020-09-25,금요일,1880095.5,1789.0,1879261.0,1011.0,2074513.0,955.0,1887.1,23095.0,...,54322.0,4178.0,312214.8,2999.0,362741.0,3357.0,479683.1,3618.0,303779.6,10844.0
1730,2020-09-26,토요일,1661090.9,1760.0,1709385.7,1075.0,2089081.2,961.0,959.0,22510.0,...,61213.0,3770.0,327395.8,3065.0,390361.2,3092.0,521493.8,3691.0,313295.7,10636.0
1731,2020-09-27,일요일,25396.0,3066.0,38222.0,1139.0,18240.0,1056.0,60.0,22333.0,...,144.0,4076.0,285.0,3707.0,2464.0,3252.0,21717.0,3567.0,9734.0,10699.0


In [7]:
def make_dataset(data, label, window_size=10):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
scaler3 = MinMaxScaler()
scaler4 = MinMaxScaler()
scaler5 = MinMaxScaler()
scaler6 = MinMaxScaler()
scaler7 = MinMaxScaler()
scaler8 = MinMaxScaler()
scaler9 = MinMaxScaler()
scaler10 = MinMaxScaler()
scaler11= MinMaxScaler()
scaler12 = MinMaxScaler()
scaler13 = MinMaxScaler()
scaler14 = MinMaxScaler()
scaler15 = MinMaxScaler()
scaler16 = MinMaxScaler()
scaler17 = MinMaxScaler()
scaler18 = MinMaxScaler()
scaler19 = MinMaxScaler()
scaler20 = MinMaxScaler()
scaler21 = MinMaxScaler()

In [9]:
model_dict = {}
split = 28 #validation

import os

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

for pum in tqdm(unique_pum + unique_kind):
    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = preprocessing1(temp_df,train, pum, 28, weather, joosan_dict, code_dict)
    temp_df = temp_df[28:]
    temp_df = temp_df.fillna(method ='ffill')
    temp_df = temp_df.fillna(method ='bfill')
    tmep_df = temp_df.dropna()
   
    train_df = temp_df
    quartile_1 = train_df.quantile(0.25)
    quartile_3 = train_df.quantile(0.75)
    IQR = quartile_3 - quartile_1
    condition =(train_df > (quartile_3 + 1.5 * IQR)) | (train_df < (quartile_1 - 1.5 * IQR))
    condition = condition.any(axis=1)
    search_df = train_df[condition]
    a = train_df.drop(search_df.index, axis=0)
    if(len(a) > 1000):
        temp_df = a  
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0].iloc[:][f'{week_num}_week']
        
        if(pum == '배추'):
            scaler = scaler1
        if(pum == '무'):
            scaler = scaler2
        if(pum == '양파'):
            scaler = scaler3
        if(pum == '건고추'):
            scaler = scaler4
        if(pum == '마늘'):
            scaler = scaler5
        if(pum == '대파'):
            scaler = scaler6
        if(pum == '얼갈이배추'):
            scaler = scaler7
        if(pum == '양배추'):
            scaler = scaler8
        if(pum == '깻잎'):
            scaler = scaler9
        if(pum == '시금치'):
            scaler = scaler10
        if(pum == '미나리'):
            scaler = scaler11
        if(pum == '당근'):
            scaler = scaler12
        if(pum == '파프리카'):
            scaler = scaler13
        if(pum == '새송이'):
            scaler = scaler14
        if(pum == '팽이버섯'):
            scaler = scaler15
        if(pum == '토마토'):
            print("good")
            scaler = scaler16
        if(pum == '청상추'):
            scaler = scaler17
        if(pum == '백다다기'):
            scaler = scaler18
        if(pum == '애호박'):
            scaler = scaler19
        if(pum == '캠벨얼리'):
            scaler = scaler20
        if(pum == '샤인마스캇'):
            scaler = scaler21
        
        
        df_scaled_x = scaler.fit_transform(x)
        df_scaled_x = pd.DataFrame(df_scaled_x)
        df_scaled_x.columns = temp_df.columns[:-3]
        x = df_scaled_x
    
        feature_cols = x.columns
        label_cols = [f'{week_num}_week']

        # train dataset
        train_feature, train_label = make_dataset(x, y, 1)

        x_train = train_feature[:-split]
        y_train = train_label[:-split]
        x_valid = train_feature[-split:]
        y_valid = train_label[-split:]
        
        model_dict[f'{pum}_model_{week_num}'] = Sequential()
        model_dict[f'{pum}_model_{week_num}'].add(LSTM(16, 
               input_shape=(train_feature.shape[1],train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )
        model_dict[f'{pum}_model_{week_num}'].add(Dense(8))
        model_dict[f'{pum}_model_{week_num}'].add(Dense(1))

        model_path = 'model'
        model_dict[f'{pum}_model_{week_num}'].compile(loss='mean_absolute_error', optimizer='SGD')
        early_stop = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights = True)
        filename = os.path.join(model_path, 'tmp_checkpoint1.h5')
        checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

        history = model_dict[f'{pum}_model_{week_num}'].fit(x_train, y_train, 
                    epochs=10000, 
                    batch_size=16,
                    validation_data=(x_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])

Output hidden; open in https://colab.research.google.com to view.